# Deep Video Analytics tutorial

### Please run this notebook on Host OS or google cloud shell with dvactl configured & DVA running

##### Make sure following modules can be imported succcessfully by running 
`pip install --upgrade requests pillow`

In [1]:
import os, requests, sys, json, jsonschema
from PIL import Image as PImage
from IPython.display import Image, display

### This tutorial uses REST API endpoint through DVA python client

In [2]:
sys.path.append('client/') #add DVA client to python path 

In [3]:
import dvaclient
import logging, json

## Using Deep Video Analytics python client

####  REST API token generated by `dvactl start`  is stored in creds.json it can also be found at /security page in DVA UI

In [4]:
creds = json.load(file('creds.json'))
server, token = creds['server'], creds['token']
# following line checks if the notebook is running "inside" the container
if os.path.abspath('.') == '/root/DVA':
    server = "http://localhost/api/"
ctx = dvaclient.context.DVAContext(server=server,token=token)

## REST API vs DVAPQL

Deep Video Analytics uses REST API (GET, LIST, but **NOT POST**) for exposing state of the system along metadata and urls to raw contents such as videos and images. **/queries is the only endpoint that supports POST requests** for mutating the state of the system. The user submits a DVAPQL query for loading videos/datasets, launching processing jobs, importing models and performing visual search. 

### List videos

In [6]:
for v in ctx.list_videos():
    print "{name} with ID: {vid}".format(name=v['name'],vid=v['id'])

crnn_test with ID: 560d70e6-be51-4b9e-b652-e2bf299fc565
exampledataset with ID: e01b4b33-6e96-457d-b1f7-720f7014396b
coco_train_ci with ID: 9a958534-10da-4b14-8ef6-a6dc91e33d68
citest with ID: ee5bfb74-adb2-4675-af14-e988e1829c2b
livestream with ID: 319fcdb0-68c1-45d9-bbf2-8f5975c8b8d8
spectre with ID: 7cead489-2ac7-414a-9dbe-79a09bcc24e5
spectre with ID: 8c319efa-d8c8-49ab-a2ad-8a223505c3b4


### List configured indexing, detection and analysis models

In [7]:
for v in ctx.list_models():
    print v['name'], v['model_type']

coco D
textbox D
face D
inception I
facenet I
crnn A
tagger A
Facenet_LOPQ_on_LFW P
Inception_PCA_from_Youtube8M P


## List events

In [8]:
_ = ctx.list_events(verbose=True)

2018-08-04T22:54:35.484379Z 2018-08-04T22:54:35.483782Z perform_init True True 0.0
2018-08-04T22:55:30.315301Z 2018-08-04T22:55:30.314943Z perform_launch True True 0.0
2018-08-04T23:10:36.588681Z 2018-08-04T23:10:36.588123Z perform_launch True True 0.0
2018-08-04T23:11:11.752733Z 2018-08-04T23:11:35.880398Z perform_indexing True True 10.367042
2018-08-04T22:55:30.385110Z 2018-08-04T22:55:53.666255Z perform_import True True 0.843314
2018-08-04T22:55:30.379033Z 2018-08-04T22:55:53.666355Z perform_import True True 0.834929
2018-08-04T22:55:54.483268Z 2018-08-04T22:55:54.515338Z perform_dataset_extraction True True 0.077662
2018-08-04T22:55:30.394052Z 2018-08-04T22:55:50.716932Z perform_process_monitoring True True 60.183073
2018-08-04T22:55:30.390894Z 2018-08-04T22:55:53.666483Z perform_import True True 1.93403
2018-08-04T22:55:54.586692Z 2018-08-04T22:55:54.616989Z perform_analysis True True 14.232323
2018-08-04T22:55:57.937029Z 2018-08-04T22:56:03.197808Z perform_indexing True True 6.51

## Process a video

In [9]:
video_processing_query = dvaclient.query.ProcessVideoURL(name="spectre",url="https://www.youtube.com/watch?v=ashLaclKCik")
video_processing_query.execute(ctx)

##### Optionally you can "wait" on it to complete,

In [ ]:
video_processing_query.wait(verbose=True)

## Finding similar images

### Query image

In [5]:
leo_query_image = 'tests/data/query_leo.png'

### Get configured retrievers and indexers

In [6]:
indexers = {r['name']:r for r in ctx.list_models() if r['model_type'] == 'I'}
retrievers = {r['name']:r for r in ctx.list_retrievers()}

### Make a query using inception retriever and inception indexer

In [7]:
inception_indexer_pk = indexers['inception']['id']
inception_retriever_pk = retrievers['inception']['id']
q = dvaclient.query.FindSimilarImages(leo_query_image,indexer_pk=inception_indexer_pk,retriever_pk=inception_retriever_pk)
q.execute(context=ctx)
q.wait()
q.gather_search_results()

Timed out after 180 seconds


KeyError: 'query_results'

### Visualize results

In [ ]:
for r in q.search_results:
    r.visualize()

In [ ]:
q = dvaclient.query.FindSimilarImages('tests/data/query_car.png',indexer_pk=inception_indexer_pk,retriever_pk=inception_retriever_pk)
q.execute(context=ctx)
q.wait()
q.gather_search_results()
for r in q.search_results:
    r.visualize()